In [720]:
%load_ext autoreload
%autoreload 2
## allow you to display your plot in the notebook
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [721]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [722]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [723]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')
train_set.columns.size, test_set.columns.size

(81, 80)

In [724]:
train_set.SalePrice = np.log(train_set.SalePrice)
# display_all(train_set.head().T)

In [725]:
train_cats(train_set)
train_cats(test_set)
train_set.columns.size, test_set.columns.size

(81, 80)

In [726]:
# display_all(train_set.isnull().sum().sort_index()/len(train_set))

In [727]:
# os.makedirs('tmp', exist_ok=True)
# train_set.to_feather('tmp/house-prices-train')
# test_set.to_feather('tmp/house-prices-test')

In [728]:
# train_set = pd.read_feather('tmp/house-prices-train')
# test_set = pd.read_feather('tmp/house-prices-test')
# train_set.columns.size, test_set.columns.size

In [729]:
# ??proc_df
# ??fix_missing

replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [730]:
# df == features && y == actuale sales prices
df, y, _ = proc_df(train_set, 'SalePrice',na_dict=nas)
test_df, _, _ = proc_df(test_set, na_dict=nas)
df.columns.size, test_df.columns.size

(91, 91)

In [731]:
# add missing columns to train set and default value as True
# cols = ['BsmtFinSF1_na','BsmtFinSF2_na','BsmtUnfSF_na','TotalBsmtSF_na','BsmtFullBath_na','BsmtHalfBath_na','GarageCars_na','GarageArea_na']
# for col in cols:
#     df[col] = True
# test_df = test_df.drop(cols,axis=1)

In [732]:
df.columns.size, test_df.columns.size

(91, 91)

In [733]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [734]:
m.score(df,y)

0.9745539890566409

In [735]:
train_set.shape, test_set.shape

((1460, 81), (1459, 80))

In [736]:
def split_vals(dataset,n): return dataset[:n].copy(), dataset[n:].copy()

n_valid = 100  # 
n_trn = len(df)-n_valid
# raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((1360, 91), (1360,), (100, 91), (100,))

In [737]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

def _print_score(m):
    res = [rmse(m.predict(df), y), m.score(df, y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [738]:
m_test = RandomForestRegressor(n_estimators=80, n_jobs=-1,  oob_score=True)
%time m_test.fit(X_train,y_train)
print_score(m_test)

CPU times: user 1.77 s, sys: 3.14 ms, total: 1.78 s
Wall time: 269 ms
[0.0566530256949054, 0.12973356148926504, 0.979977407217676, 0.8861648397579088, 0.8660541975331053]


In [739]:
m = RandomForestRegressor(n_estimators=80, min_samples_leaf=3,max_features=0.5, n_jobs=-1, oob_score=True)
%time m.fit(X_train,y_train)
print_score(m)

CPU times: user 801 ms, sys: 8.92 ms, total: 810 ms
Wall time: 171 ms
[0.07624234280058594, 0.12476998918131631, 0.9637367635290625, 0.8947088142003574, 0.875550865644395]


In [740]:
predictions = np.exp(m.predict(test_df))

In [654]:
s = pd.DataFrame({'Id':test_df['Id'],'SalePrice':predictions})
s.to_csv('house_prices_final.csv',index=False)